<a href="https://colab.research.google.com/github/sakinakagawa/ai_2022_webapp2/blob/main/ai_webapp4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install icrawler

from icrawler.builtin import BingImageCrawler

# ①石原さとみ×100
crawler = BingImageCrawler(storage={"root_dir": "石原さとみ"})
crawler.crawl(keyword="石原さとみ", max_num=100)

from icrawler.builtin import BingImageCrawler

# ②新垣結衣×100
crawler = BingImageCrawler(storage={"root_dir": "新垣結衣"})
crawler.crawl(keyword="新垣結衣", max_num=100)

from icrawler.builtin import BingImageCrawler

# ⑤北川景子×100
crawler = BingImageCrawler(storage={"root_dir": "北川景子"})
crawler.crawl(keyword="北川景子", max_num=100)

from icrawler.builtin import BingImageCrawler

# ⑥小松菜奈×100
crawler = BingImageCrawler(storage={"root_dir": "小松菜奈"})
crawler.crawl(keyword="小松菜奈", max_num=100)

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


ERROR:downloader:Response status code 403, file https://storage.mainichikirei.jp/images/2018/10/25/20181025dog00m100010000c/001_size9.jpg
ERROR:downloader:Response status code 403, file https://storage.mantan-web.jp/images/2016/10/04/20161004dog00m200023000c/001_size9.jpg
ERROR:downloader:Exception caught when downloading file https://www.sponichi.co.jp/entertainment/news/2020/07/17/jpeg/20200716s00041000311000p_view.jpg, error: HTTPSConnectionPool(host='www.sponichi.co.jp', port=443): Read timed out. (read timeout=5), remaining retry times: 2
ERROR:downloader:Exception caught when downloading file https://www.sponichi.co.jp/entertainment/news/2020/07/17/jpeg/20200716s00041000311000p_view.jpg, error: HTTPSConnectionPool(host='www.sponichi.co.jp', port=443): Read timed out. (read timeout=5), remaining retry times: 1
ERROR:downloader:Exception caught when downloading file https://www.sponichi.co.jp/entertainment/news/2020/07/17/jpeg/20200716s00041000311000p_view.jpg, error: HTTPSConnecti

In [2]:
from PIL import Image
import os, glob
import numpy as np
from PIL import ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True

classes = ["石原さとみ", "新垣結衣", "北川景子", "小松菜奈"]
num_classes = len(classes)
image_size = 64
num_testdata = 25

X_train = []
X_test  = []
y_train = []
y_test  = []

for index, classlabel in enumerate(classes):
    photos_dir = "./" + classlabel
    files = glob.glob(photos_dir + "/*.jpg")
    for i, file in enumerate(files):
        image = Image.open(file)
        image = image.convert("RGB")
        image = image.resize((image_size, image_size))
        data = np.asarray(image)
        if i < num_testdata:
            X_test.append(data)
            y_test.append(index)
        else:

            for angle in range(-20, 20, 5):

                img_r = image.rotate(angle)
                data = np.asarray(img_r)
                X_train.append(data)
                y_train.append(index)
                img_trains = img_r.transpose(Image.FLIP_LEFT_RIGHT)
                data = np.asarray(img_trains)
                X_train.append(data)
                y_train.append(index)

X_train = np.array(X_train)
X_test  = np.array(X_test)
y_train = np.array(y_train)
y_test  = np.array(y_test)

xy = (X_train, X_test, y_train, y_test)
np.save("./女優.npy", xy)

/usr/local/lib/python3.8/dist-packages/numpy/lib/npyio.py:528: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  arr = np.asanyarray(arr)


In [3]:
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Activation, Dropout, Flatten, Dense
from keras.optimizers import RMSprop 



from keras.utils import np_utils
import keras
import numpy as np

classes = ["石原さとみ", "新垣結衣", "北川景子", "小松菜奈"]
num_classes = len(classes)
image_size = 64


def load_data():
    X_train, X_test, y_train, y_test = np.load("./女優.npy", allow_pickle=True)
    X_train = X_train.astype("float") / 255
    X_test  = X_test.astype("float") / 255
    y_train = np_utils.to_categorical(y_train, num_classes)
    y_test  = np_utils.to_categorical(y_test, num_classes)

    return X_train, y_train, X_test, y_test


def train(X, y, X_test, y_test):
    model = Sequential()


    model.add(Conv2D(32,(3,3), padding='same',input_shape=X.shape[1:]))
    model.add(Activation('relu'))
    model.add(Conv2D(32,(3,3)))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2,2)))
    model.add(Dropout(0.1))

    model.add(Conv2D(64,(3,3), padding='same'))
    model.add(Activation('relu'))
    model.add(Conv2D(64,(3,3)))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2,2)))
    model.add(Dropout(0.25))

    model.add(Flatten())
    model.add(Dense(512))
    model.add(Activation('relu'))
    model.add(Dropout(0.45))
    model.add(Dense(4)) 
    model.add(Activation('softmax'))

    opt = RMSprop(lr=0.00005, decay=1e-6)
    model.compile(loss='categorical_crossentropy',optimizer=opt,metrics=['accuracy'])
    model.fit(X, y, batch_size=28, epochs=40)
    model.save('./cnn.h5')

    return model

def main():
    X_train, y_train, X_test, y_test = load_data()

    model = train(X_train, y_train, X_test, y_test)

main()

/usr/local/lib/python3.8/dist-packages/keras/optimizers/optimizer_v2/rmsprop.py:135: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(RMSprop, self).__init__(name, **kwargs)


Epoch 1/40
63/63 [==============================] - 11s 15ms/step - loss: 1.0756 - accuracy: 0.4352
Epoch 2/40
63/63 [==============================] - 1s 13ms/step - loss: 0.9450 - accuracy: 0.5315
Epoch 3/40
63/63 [==============================] - 1s 13ms/step - loss: 0.8533 - accuracy: 0.5975
Epoch 4/40
63/63 [==============================] - 1s 13ms/step - loss: 0.7620 - accuracy: 0.6439
Epoch 5/40
63/63 [==============================] - 1s 13ms/step - loss: 0.6942 - accuracy: 0.6932
Epoch 6/40
63/63 [==============================] - 1s 13ms/step - loss: 0.6409 - accuracy: 0.7179
Epoch 7/40
63/63 [==============================] - 1s 13ms/step - loss: 0.5755 - accuracy: 0.7546
Epoch 8/40
63/63 [==============================] - 1s 13ms/step - loss: 0.5306 - accuracy: 0.7964
Epoch 9/40
63/63 [==============================] - 1s 13ms/step - loss: 0.4738 - accuracy: 0.8114
Epoch 10/40
63/63 [==============================] - 1s 13ms/step - loss: 0.4399 - accuracy: 0.8337
Epoch 11

In [5]:
import keras
import sys, os
import numpy as np
from PIL import Image
from keras.models import load_model

imsize = (64, 64)

testpic     = "./画像.jpg"
keras_param = "./cnn.h5"

def load_image(path):
    img = Image.open(path)
    img = img.convert('RGB')
    img = img.resize(imsize)
    img = np.asarray(img)
    img = img / 255.0
    return img

model = load_model(keras_param)
img = load_image(testpic)
prd = model.predict(np.array([img]))
print(prd)
prelabel = np.argmax(prd, axis=1)
if prelabel == 0:
    print(">>> 石原さとみ")
elif prelabel == 1:
    print(">>> 新垣結衣")
elif prelabel == 2:
    print(">>> 北川景子")
elif prelabel == 3:
    print(">>> 小松菜奈")

1/1 [==============================] - 0s 95ms/step
[[1.6818819e-07 7.4633710e-02 1.8325117e-13 9.2536616e-01]]
>>> 小松菜奈
